# AAT data check | 300264086 = Associated Concepts Facet
- 2024-05-18
- Language check on Getty ULAN data acquired via SPARQL-endpoint
- V. Martens

# Import

## Import packages

In [3]:
# handling jsons
import json
from json.decoder import JSONDecodeError

# spell check module
import language_tool_python

# creating time stamps
# from datetime import datetime
import time

# importing files
import glob
import os

# progress bar
from tqdm.notebook import tqdm

# regex module
import re

# for multi-threading
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

# data wrangling
import pandas as pd
import numpy as np

# back up files
import pickle

# check for western spellings
from alphabet_detector import AlphabetDetector

# spacy lang detect
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

from neg_filter_aat import *

# preferences
# adjust pandas to show all cols
pd.set_option('display.max_colwidth', None)

## Import data

### LOD results

In [2]:
# load lod-query from getty_ulan_sparql_endpoint script
%store -r results_df

# to create a more standardized naming
df_results = results_df.copy()

results_df.shape

(12983, 8)

## Constants

In [4]:
# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_errors = f'{time_stamp}_df_errors_aat.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_lod_results = f'{time_stamp}_df_lod_results_aat.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_excel_export = f'{time_stamp}_spell_check_AAT_300264086.xlsx'

print(f"{filename_df_errors}, {filename_df_lod_results}, {filename_excel_export}")

20240602-111025_df_errors_aat.pickle, 20240602-111025_df_lod_results_aat.pickle, 20240602-111025_spell_check_AAT_300264086.xlsx


## Import functions

In [5]:
@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()

def check_df_row(df, col, i):
    '''
    '''
    dict_source_matches = {}
    try:
        matches = tool.check(df[col].iloc[i])
        dict_source_matches[df['Subject.value'].iloc[i]] = matches 
        return dict_source_matches
    except (JSONDecodeError, NameError) as e:
        return dict_source_matches[df['Subject.value'].iloc[i]] == 'na'
        print(e, df['Subject.value'].iloc[i])
    
    
def parse_list_of_jsons(json_list: list) -> pd.DataFrame():
    '''doc string'''
    df_errors = pd.DataFrame()

    for item in json_list:

        for k, v in item.items():
            source = k 

            for item in (v):

                df_error = pd.DataFrame(
                    (
                             source,
                             item.ruleId,
                             item.message,
                             item.replacements,
                             item.offsetInContext,
                             item.context,
                             item.offset,
                             item.errorLength,
                             item.category,
                             item.ruleIssueType,
                             item.sentence,
                             item.context[item.offsetInContext:int(item.offsetInContext + item.errorLength)],
                            )
                ).T

                df_errors = pd.concat([df_errors, df_error])

    df_errors = df_errors.rename(columns={
              0 : "url",
              1 : 'ruleId',
              2 : 'message', 
              3 : 'replacements',
              4 : 'offsetInContext',
              5 : 'context',
              6 : 'offset', 
              7 : 'errorLength',
              8 : 'category', 
              9 : 'ruleIssueType', 
              10 : 'sentence',
              11 : 'misspelledWord'})          

    return df_errors

def load_latest_file(filepath:str) -> str:
    '''args: string with filepath
    returns: latest file'''
    list_of_files = glob.glob(filepath)
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    return latest_file

def create_aat_weblink(string:str) -> str:
    '''from a lod-url creates a regular ulan webpage link
    args: ulan lod landing page
    returns: regular human readable webpage
    '''
    aat_regex = re.compile('\d+')
    match_class_object = re.search(aat_regex, string)
    aat_id = match_class_object.group(0)
    return f"https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid={aat_id}"

def check_for_western_chars(string):
    try:
        valids = re.match(r"[A-Za-z-]+", string).group(0)
        return valids
    except:
        pass

def check_for_capitalization(string):
    try:
        valids = re.match(r"^[A-Z]", string).group(0)
        return valids
    except:
        return np.nan

# Manipulate data

In [310]:
cpu_count = mp.cpu_count()
cpu_count

8

In [311]:
results_df = results_df[(results_df['ScopeNote.value'].notnull())]
results_df.shape

NameError: name 'results_df' is not defined

In [7]:
results_df.head()

,Subject.type,Subject.value,ScopeNote.xml:lang,ScopeNote.type,ScopeNote.value,Term.xml:lang,Term.type,Term.value
0,uri,http://vocab.getty.edu/aat/300189559,en,literal,Referring to the sex that in reproduction normally produces sperm cells or male gametes.,en,literal,male
1,uri,http://vocab.getty.edu/aat/300189557,en,literal,Referring to the sex that normally produces eggs or female germ cells.,en,literal,female
2,uri,http://vocab.getty.edu/aat/300451703,en,literal,"Motility or other conditions that limit a person's movements, senses, or activities in a conventional sense.",en,literal,physical disabilities
3,uri,http://vocab.getty.edu/aat/300266528,en,literal,"Persian wool carpets made in Herat, characterized by a wine red field color, and a border of clear emerald green and asymmetrical knotting; often made of silk.",en,literal,Herat carpets
4,uri,http://vocab.getty.edu/aat/300056330,en,literal,"Pictorial narrative device featuring two or more successive actions from the same story or related stories within one setting. Term may be confused with ""simultaneous representation,"" which has to do with point of view, not with sequential episodes depicted in one scene.",en,literal,continuous narration


In [309]:
%%time

from tqdm import tqdm

# instance of language_tool_python
tool = language_tool_python.LanguageTool('en-US')

# Start pool
thread_pool = ThreadPoolExecutor(max_workers=cpu_count, thread_name_prefix = 'Thread')

# reate futures
futures = [thread_pool.submit(check_df_row, results_df, 'ScopeNote.value', i) for i in tqdm(range(len(results_df)), total=len(results_df), desc='building futures')]

# submit tasks
results = [future.result() for future in tqdm(futures, total=len(futures), desc='spell check data')]

# # Changed the func to add a df[col], to make it more compatible
# # instance of language_tool_python
# tool = language_tool_python.LanguageTool('en-US')

# # Start pool
# thread_pool = ThreadPoolExecutor(max_workers=cpu_count, thread_name_prefix = 'Thread')

# # reate futures
# futures = [thread_pool.submit(check_df_row, results_df, i) for i in tqdm(range(len(results_df)) , total=len(results_df), desc='building futures')]

# # submit tasks
# results = [future.result() for future in tqdm(futures, total=len(futures), desc='spell check data')]

# results = []
# for future in tqdm(futures, total=len(futures), desc='spell check data'):
#     try:
#         results.append(future.result())
#     except (JSONDecodeError, NameError) as e:
#         print(e)
#         pass

# # close tool
# tool.close()

NameError: name 'cpu_count' is not defined

In [ ]:
df_errors = parse_list_of_jsons(results)

# Back up as a pickle

##  Write

In [19]:
with open(f'data_dumps/{filename_df_errors}', 'wb') as handle:
    pickle.dump(df_errors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'data_dumps/{filename_df_lod_results}', 'wb') as handle:
    pickle.dump(results_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'df_errors' is not defined

## Load

In [6]:
latest_picke_file = load_latest_file('data_dumps/*lod_results_aat.pickle')

# Open the file in binary mode
with open(latest_picke_file, 'rb') as file:
      
    # Call load method to deserialze
    df_lod_results_backup = pickle.load(file)
    
df_lod_results_backup.shape

data_dumps\20240530-221803_df_lod_results_aat.pickle


(10242, 8)

In [28]:
latest_picke_file = load_latest_file('data_dumps/*errors_aat.pickle')

# Open the file in binary mode
with open(latest_picke_file, 'rb') as file:
      
    # Call load method to deserialze
    df_error_backup = pickle.load(file)
    
df_error_backup.shape

data_dumps\20240530-221803_df_errors_aat.pickle


(5759, 12)

# Manipulate data

In [29]:
# creates human readable ulan link, based on lod link
df_error_backup['url_aat'] = df_error_backup['url'].apply(create_aat_weblink)

# Filter out spelling mistakes

## Subsetting based on ruleId

In [30]:
# filter out punctuation errors
df_punctuation = df_error_backup[(df_error_backup['ruleId'] == 'UPPERCASE_SENTENCE_START') | 
                                 (df_error_backup['ruleId'] == 'COMMA_PARENTHESIS_WHITESPACE') |
                                 (df_error_backup['ruleId'] == 'WHITESPACE_RULE') | 
                                 (df_error_backup['ruleId'].str.contains('EN_COMPOUNDS') == True) | 
                                 (df_error_backup['ruleId'].str.contains('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA') == True) | 
                                 (df_error_backup['ruleId'].str.contains('MISSING_COMMA_AFTER_INTRODUCTORY_PHRASE') == True) | 
                                 (df_error_backup['ruleId'] == 'SENTENCE_WHITESPACE' ) | 
                                 (df_error_backup['ruleId'] == 'ETC_PERIOD') | 
                                 (df_error_backup['ruleId'] == 'LC_AFTER_PERIOD') | # punctuation
                                 (df_error_backup['ruleId'] == 'EN_DIACRITICS_REPLACE_DECOLLETAGE') | # 
                                 (df_error_backup['ruleId'] == 'EN_DIACRITICS_REPLACE_CREPE') |  # punctuation
                                 (df_error_backup['ruleId'] == 'FILE_EXTENSIONS_CASE')] # caps


df_agreement = df_error_backup[((df_error_backup['ruleId'] == 'AGREEMENT_SENT_START') & (df_error_backup['misspelledWord'] == 'remains')) |
                               ((df_error_backup['ruleId'] == 'AGREEMENT_SENT_START') & (df_error_backup['misspelledWord'] == 'locks')) |
                               ((df_error_backup['ruleId'] == 'AGREEMENT_SENT_START') & (df_error_backup['misspelledWord'] == 'is'))]

# filter out grammatical errors
df_grammar = df_error_backup[(df_error_backup['ruleId'].str.contains('DEPEND_ON') == True) |
                             (df_error_backup['ruleId'].str.contains('FEWER_LESS') == True) |
                             (df_error_backup['ruleId'].str.contains('ENGLISH_WORD_REPEAT_BEGINNING_RULE') == True) |
                             (df_error_backup['ruleId'].str.contains('EN_DIACRITICS_REPLACE_CREPE') == True) |
                             (df_error_backup['ruleId'] == 'FOR_FRO') |
                             (df_error_backup['ruleId'] == 'SUBJECT_VERB_AGREEMENT_PLURAL') |
                             (df_error_backup['ruleId'] == 'NODT_DOZEN') |
                             (df_error_backup['ruleId'] == 'TO_NON_BASE') |
                             (df_error_backup['ruleId'] == 'AS_IS_VBG') |
                             (df_error_backup['ruleId'] == 'MD_BASEFORM') |
                             (df_error_backup['ruleId'] == 'THROUGH_THOROUGH') | # not a false hit, but needs to be throughflow, instead of 'through flow'
                             (df_error_backup['ruleId'] == 'MANY_KINDS_OF') |
                             (df_error_backup['ruleId'] == 'BE_VBP_IN') |
                             (df_error_backup['ruleId'] == 'AFTERWARDS_US') |
                             (df_error_backup['ruleId'] == 'NNS_THAT_VBZ') |
                             (df_error_backup['ruleId'] == 'PLURAL_MODIFIER') |
                             (df_error_backup['ruleId'] == 'PRP_PAST_PART') |
                             (df_error_backup['ruleId'] == 'EVERYDAY_EVERY_DAY') |
                             (df_error_backup['ruleId'] == 'ENGLISH_WORD_REPEAT_BEGINNING_RULE') |
                             (df_error_backup['ruleId'] == 'FOR_FRO') |
                             (df_error_backup['ruleId'] == 'SUBJECT_VERB_AGREEMENT_PLURAL') |
                             (df_error_backup['ruleId'] == 'ARTICLE_ADJECTIVE_OF') |
                             (df_error_backup['ruleId'] == 'EN_A_VS_AN') |
                             (df_error_backup['ruleId'] == 'POSSESSIVE_APOSTROPHE') |
                             (df_error_backup['ruleId'] == 'THE_SUPERLATIVE') |
                             (df_error_backup['ruleId'] == 'PLURAL_THAT_AGREEMENT')]

# subsetting on false hits, and historical left-over
df_error_backup = df_error_backup[(df_error_backup['ruleId'] != 'UPPERCASE_SENTENCE_START') &
                                  (df_error_backup['ruleId'] != 'COMMA_PARENTHESIS_WHITESPACE') & 
                                  (df_error_backup['ruleId'] != 'WHITESPACE_RULE') &
                                  (df_error_backup['ruleId'].str.contains('EN_COMPOUNDS') == False) & 
                                  (df_error_backup['ruleId'].str.contains('SENT_START_CONJUNCTIVE_LINKING_ADVERB_COMMA') == False) &
                                  (df_error_backup['ruleId'].str.contains('MISSING_COMMA_AFTER_INTRODUCTORY_PHRASE') == False) & 
                                  (df_error_backup['ruleId'] != 'SENTENCE_WHITESPACE' ) &
                                  (df_error_backup['ruleId'].str.contains('DEPEND_ON') == False) &
                                  (df_error_backup['ruleId'] != 'ETC_PERIOD') & 
                                  (df_error_backup['ruleId'] != 'LC_AFTER_PERIOD') & # punctuation
                                  (df_error_backup['ruleId'] != 'EN_DIACRITICS_REPLACE_DECOLLETAGE') & # punctuation
                                  (df_error_backup['ruleId'] != 'EN_DIACRITICS_REPLACE_CREPE') & # punctuation
                                  (df_error_backup['ruleId'] != 'FILE_EXTENSIONS_CASE') & # caps
                                  (df_error_backup['ruleId'].str.contains('RED_READ') == False) & # false hit
                                  (df_error_backup['ruleId'] != 'RITE_WRITE') & # false hit
                                  (df_error_backup['ruleId'] != 'FROM_FORM') &  # false hit
                                  (df_error_backup['ruleId'] != 'BESIDES_BESIDE') &  # false hit
                                  (df_error_backup['ruleId'] != 'NEITHER_NOR') & # false hit
                                  (df_error_backup['ruleId'] != 'OTHER_OTHERS') & # false hit
                                  (df_error_backup['ruleId'] != 'ITS_TO_IT_S') & # false hit
                                  (df_error_backup['ruleId'] != 'IT_IS_JJ_TO_VBG') & # false hit
                                  (df_error_backup['ruleId'] != 'CANT') & 
                                  (df_error_backup['ruleId'] != 'AGREEMENT_SENT_START' )] # false hit

# randomly found errors
df_random_finds =  df_error_backup[(df_error_backup['misspelledWord'] == 'Scandanavian') |
                                   (df_error_backup['misspelledWord'] == 'Piemontese')]

df_floor = df_error_backup[(df_error_backup['misspelledWord'] == 'storey' ) |  # floor in us english
                           (df_error_backup['misspelledWord'] == 'storeys' )]


# if ruleId not in df_other_errors and not in df_random_finds
df_error_backup = df_error_backup[(~df_error_backup['ruleId'].isin(df_grammar['ruleId'])) & 
                                  (~df_error_backup['ruleId'].isin(df_punctuation['ruleId']))]

df_error_backup = df_error_backup[(~df_error_backup['misspelledWord'].isin(df_random_finds['misspelledWord']))]

print(f"grammar mistakes: {df_grammar.shape[0]}\n agreement errors: {df_agreement.shape[0]} \n punctuation mistakes {df_punctuation.shape[0]} \n other errors: {df_error_backup.shape[0]} \n floors: {df_floor.shape[0]} \n random finds: {df_random_finds.shape[0]}")

grammar mistakes: 80
 agreement errors: 3 
 punctuation mistakes 159 
 other errors: 5501 
 floors: 13 
 random finds: 2


# Filter out foreign hits

In [31]:
# filters out sentences with most likely foreign words 
df_foreign = df_error_backup[(df_error_backup['sentence'].str.lower().str.contains('indian') == True) |
                             (df_error_backup['sentence'].str.lower().str.contains('arabic') == True) |
                             (df_error_backup['sentence'].str.lower().str.contains('chinese') == True) |
                             (df_error_backup['sentence'].str.lower().str.contains('persian') == True) |
                             (df_error_backup['sentence'].str.lower().str.contains('nahuatl') == True) ]

# checked subset for hits and subset those into a df
df_foreign_positive = df_foreign[(df_foreign['misspelledWord'].str.contains('mould') == True) | 
                                 (df_foreign['misspelledWord'].str.contains('storey') == True) |
                                 (df_foreign['misspelledWord'] == 'doorframe') |
                                 (df_foreign['misspelledWord'] == 'coffered') |
                                 (df_foreign['misspelledWord'] == 'doorsill') |
                                 (df_foreign['misspelledWord'] == 'oversailing') |
                                 (df_foreign['misspelledWord'] == 'atlantid') |
                                 (df_foreign['misspelledWord'] == '\'')]

# filters out sentences with most likely foreign words
df_error_backup = df_error_backup[(df_error_backup['sentence'].str.lower().str.contains('indian') == False) & 
                                  (df_error_backup['sentence'].str.lower().str.contains('arabic') == False) &
                                  (df_error_backup['sentence'].str.lower().str.contains('chinese') == False) &
                                  (df_error_backup['sentence'].str.lower().str.contains('persian') == False) & 
                                  (df_error_backup['sentence'].str.lower().str.contains('nahuatl') == False)]

print(f"other errors: {df_error_backup.shape[0]}, \n foreign errors: {df_foreign_positive.shape[0]}")

other errors: 4386, 
 foreign errors: 116


## Filter out frequently occuring mismatches

In [32]:
df_error_backup = negative_aat_filter(df_error_backup)
print(f"other errors: {df_error_backup.shape[0]}")

other errors: 2531


## Filter out words that start with a capital

In [33]:
# run regex func
df_error_backup['capital_start'] = df_error_backup['misspelledWord'].apply(check_for_capitalization)

df_capitals = df_error_backup[(df_error_backup['capital_start'].notnull())]

df_capitals_positive = df_capitals[(df_capitals['misspelledWord'] == 'Bookbindings' ) |
                                   (df_capitals['misspelledWord'] == 'Web site' ) |
                                   (df_capitals['misspelledWord'] == 'Catholic church' ) |
                                   (df_capitals['misspelledWord'] == 'Eastern Orthodox church' ) |
                                   (df_capitals['misspelledWord'] == 'It dark' ) |
                                   (df_capitals['misspelledWord'] == 'With the exception of' ) |
                                   (df_capitals['misspelledWord'] == 'Finno-Ugric' ) | 
                                   (df_capitals['misspelledWord'] == 'Syrian desert' ) ]

df_error_backup = df_error_backup[(df_error_backup['capital_start'].isnull())]
del df_error_backup['capital_start']

print(f"other errors: {df_error_backup.shape[0]}, \n {df_capitals_positive.shape[0]}")


other errors: 1254, 
 23


# Check for language

In [34]:
# subsets chars from non western chars
df_error_backup['western_chars'] = df_error_backup['misspelledWord'].apply(check_for_western_chars)

# see if length is equal otherwise drop non-western chars
df_error_backup = df_error_backup[(df_error_backup['western_chars'].str.len() == df_error_backup['misspelledWord'].str.len())]

print(f"other errors: {df_error_backup.shape[0]}")

other errors: 546


## Check for non-Western typescripts

In [35]:
ad = AlphabetDetector()
df_error_backup['western_typo'] = df_error_backup['misspelledWord'].apply(lambda x: ad.is_latin(x))

print(f"non-western chars: {df_error_backup[(df_error_backup['western_typo'] == False )].shape[0]}")
df_error_backup = df_error_backup[(df_error_backup['western_typo'] == True )]

del df_error_backup['western_typo']

print(f"other errors: {df_error_backup.shape[0]}")

non-western chars: 0
other errors: 546


In [692]:
# x = pd.DataFrame(df_error_backup['misspelledWord'].value_counts().iloc[95:100]).reset_index()
# list_error = x['misspelledWord'].to_list()
# print(list_error)

# df_error_backup[(df_error_backup['misspelledWord'].isin(list_error))]

In [48]:
# for item in manual:
    # print(f"                (df_error_backup['misspelledWord'] != '{item}' ) &")

## Lang detect Spacy

In [758]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 2000000
nlp.add_pipe('language_detector', last=True)

# initiate master df
df_spacies = pd.DataFrame()

# run over words
for item in tqdm(df_error_backup['misspelledWord'].tolist()):
    
    # store data in doc
    doc = nlp(str(item))
    detect_language = doc._.language

    # cast and store results into variable
    word = str(doc)
    language = str(detect_language['language'])
    score =  float(detect_language['score'])

    # add to df
    df_spacy = pd.DataFrame((word, language, score)).T
    df_spacies = pd.concat([df_spacy, df_spacies])

# manipulate all spacy results
df_spacies = (df_spacies
             .rename(columns={0: 'word',
                              1: 'language',
                              2: 'score'})
             .sort_values('score', ascending=False)
             .groupby(by=['word', 'language'])
             .first()
             .reset_index())

# subset false hits
df_spacies = df_spacies[(df_spacies['word'].str.len() > 3)]

# merge back with main df
df_error_backup = pd.merge(df_error_backup, 
                           df_spacies, 
                           left_on='misspelledWord', 
                           right_on='word', 
                           how='left')

100%|██████████| 1381/1381 [00:28<00:00, 48.23it/s]


# Check for entities

In [36]:
# run spacy on trained name data
nlp = spacy.load("en_core_web_lg")

entity_list = []

# run over annotations
for i in tqdm(df_error_backup['misspelledWord'].tolist()):
    # store data in doc
    doc = nlp(str(i))
    
    # retrieve entities from doc and ad it to a list
    for entity in doc.ents:
#         print(entity.text, entity.label_)
        entity_list.append((i, entity.text, entity.label_))
    
# untuple the list    
untupled = pd.DataFrame([[y for y in  x] for x in entity_list])

# add col names
untupled = untupled.rename(columns={0:'word',
                                    1:'spacy_word',
                                    2:'entity_label'})

# merge back with main df
df_error_backup = pd.merge(df_error_backup, 
                           untupled, 
                           left_on='misspelledWord', 
                           right_on='word', 
                           how='left')

  0%|          | 0/546 [00:00<?, ?it/s]

# Merge all errors

In [39]:
# merge
df_spell_check = pd.concat([df_agreement, df_error_backup, df_grammar, df_punctuation, df_random_finds, df_floor, df_foreign_positive, df_capitals_positive])

# select cols
df_spell_check = df_spell_check[['url', 
                                 'url_aat', 
                                 'ruleId', 
                                 'message', 
                                 'sentence',
                                 'misspelledWord',
                                 'replacements', 
                                #  'offsetInContext',
#                                  'context', 
                                 'offset', 
                                 'errorLength', 
                                 'category', 
                                 'ruleIssueType',
                                ]]

# df_spell_check.drop_duplicates(keep='first', inplace=True)

# rename cols
df_spell_check = df_spell_check.rename(columns={'url' : 'url_lod_aat'})

# reset index
df_spell_check = df_spell_check.reset_index().drop(columns='index')

print(f"errors found: {df_spell_check.shape[0]}")

errors found: 966


In [40]:
df_spell_check

,url_lod_aat,url_aat,ruleId,message,sentence,misspelledWord,replacements,offset,errorLength,category,ruleIssueType
0,http://vocab.getty.edu/aat/300264460,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300264460,AGREEMENT_SENT_START,You should probably use “remain”.,Both factions remains powerful and popular today and have their main temples in Kyoto.,remains,[remain],798,7,GRAMMAR,grammar
1,http://vocab.getty.edu/aat/300225150,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300225150,AGREEMENT_SENT_START,You should probably use “lock”.,"Those parts of firearm locks that engage the hammer to hold it at full or half cock, until released by the trigger.",locks,[lock],23,5,GRAMMAR,grammar
2,http://vocab.getty.edu/aat/300265218,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300265218,AGREEMENT_SENT_START,You should probably use: “are”.,"The terms is sometimes translated as flavor, mood, or aesthetic delight.",is,[are],125,2,GRAMMAR,grammar
3,http://vocab.getty.edu/aat/300451409,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300451409,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"Time that is measured by direct observation of the Sun, by a sundial, astrolab, or directly by another device.",astrolab,[astrolabe],70,8,TYPOS,misspelling
4,http://vocab.getty.edu/aat/300446753,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300446753,MORFOLOGIK_RULE_EN_US,Possible spelling mistake. ‘moulding’ is British English.,"A type of kaṇṭha, or recessed moulding; recessed moulding in adhiṣṭhāna.",moulding,[molding],30,8,TYPOS,misspelling
...,...,...,...,...,...,...,...,...,...,...,...
961,http://vocab.getty.edu/aat/300417781,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300417781,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,Bookbindings having raised letters or designs on the surface.,Bookbindings,"[Bookbinding, Book bindings]",0,12,TYPOS,misspelling
962,http://vocab.getty.edu/aat/300400855,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300400855,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"Pre-Christian and pre-Islamic religious beliefs and practices of the Finno-Ugric peoples, who inhabit regions of northern Scandinavia, Siberia, the Baltic area, and central Europe.",Finno-Ugric,"[Finn-Ugric, Finno-Uric]",69,11,TYPOS,misspelling
963,http://vocab.getty.edu/aat/300417756,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300417756,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"Bookbindings that use leather, generally either as the primary cover or to wrap a board cover.",Bookbindings,"[Bookbinding, Book bindings]",0,12,TYPOS,misspelling
964,http://vocab.getty.edu/aat/300417741,https://www.getty.edu/vow/AATFullDisplay?find=&logic=AND&note=&subjectid=300417741,MORFOLOGIK_RULE_EN_US,Possible spelling mistake found.,"Bookbindings decorated with painting, often along with gilding or other decoration.",Bookbindings,"[Bookbinding, Book bindings]",0,12,TYPOS,misspelling


# Export

In [345]:
df_spell_check.to_excel('data_dumps\\' + filename_excel_export, index=False)

In [303]:
df_doubles = df_spell_check.groupby(['url_aat', 'offsetInContext', 'misspelledWord']).size().reset_index().rename(columns={0 : 'freq'})

In [313]:
test = pd.merge(df_spell_check, df_doubles, on=['url_aat', 'misspelledWord'], how='left')
test['freq'] = test['freq'].fillna(0)

In [286]:
# test.head()

In [323]:
# errori = test[(test['freq'] > 1)]

# test = test[(test['freq'] < 1)]
# test.shape

In [324]:
# errori

In [325]:
# errori = errori.groupby(['url_aat', 'offsetInContext', 'misspelledWord']).first()
# errori_shape

In [301]:
errori.shape

(78, 10)

In [326]:
# df_doubles = df_doubles[(df_doubles['freq'] > 1)]
# df_doubles

# Export

In [206]:
false_positives = list(set(untupled['person'].tolist()))

In [698]:
dftest = df_error_backup[(~df_error_backup['misspelledWord'].isin(false_positives)) & 
                         (df_error_backup['language'] == 'en')]

In [699]:
dftest.shape

(207, 17)

In [732]:
dftest['misspelledWord'].value_counts().iloc[40:80]

misspelledWord
righ-hand           1
chape               1
orthogonals         1
thorugh             1
helically           1
microform           1
showerheads         1
constucted          1
overflowings        1
aboveground         1
pharmaceutically    1
ploughing           1
lowlying            1
recyclability       1
monitary            1
ormamented          1
seachannels         1
fortications        1
fourescent          1
cracks              1
stylograph          1
hardwearing         1
activties           1
rushlights          1
shoreward           1
behaviour           1
mouthes             1
thetan              1
undissected         1
characers           1
freemasons          1
thaler              1
screwheads          1
synthesising        1
associatively       1
inclosed            1
preisthood          1
lockforming         1
countershed         1
heros               1
Name: count, dtype: int64

In [64]:
# dftest[(dftest['language'] == 'en')]

## Filter out cities

In [711]:
df_spacies[(df_spacies['language'] != 'en')].sample(50)

,word,language,score
55,applicaiton,it,0.999994
584,lacemaking,tl,0.857137
1134,urens,fr,0.857139
456,globose,sl,0.999994
786,petunidin,et,0.571426
62,archivolts,it,0.999997
1112,uan-yang,id,0.999997
59,arcadian,es,0.999992
367,dvinity,lt,0.999998
452,ghanta,id,0.707238


In [718]:
# df_holland = df_error_backup[(df_error_backup['sentence'].str.contains('Holland') == True)]

# # creates human readable ulan link, based on lod link
# df_holland['url_ulan'] = df_holland['x.value'].apply(create_ulan_weblink)

# the one errouneous holland
# df_holland = df_holland[(df_holland['url_ulan'].str.contains('500256837') == True)]

In [ ]:
# add cols to allign with other cols
df_holland['ruleId'] = 'PREFERRED SPELLING'
df_holland['message'] = 'Prefered spelling is the Netherlands.'
df_holland['replacements'] = ['The Netherlands']
df_holland['offsetInContext'] = 0
df_holland['context'] = df_holland['bio.value']
df_holland['offset'] = 0
df_holland['errorLength'] = 0
df_holland['category'] = 'PREFERRED SPELLING'
df_holland['ruleIssueType'] = 'typographical'
df_holland['sentence'] = df_holland['bio.value']
df_holland['misspelledWord'] = 'Holland'

# rename cols
df_holland = df_holland.rename(columns={'x.value':'url'})

# select cols
df_holland = df_holland[['url', 'ruleId', 'message', 'replacements', 'offsetInContext',
       'context', 'offset', 'errorLength', 'category', 'ruleIssueType',
       'sentence', 'misspelledWord', 'url_ulan']]

# Merge all found errors and create export

In [ ]:
# merge
df_spell_check = pd.concat([df_checked_errors, df_punctuation, df_holland])

# select cols
df_spell_check = df_spell_check[['url', 
                                 'url_ulan', 
                                'sentence',
                                 'ruleId', 
                                 'message', 
                                 'replacements', 
                                 'offsetInContext',
#                                  'context', 
                                 'offset', 
                                 'errorLength', 
                                 'category', 
                                 'ruleIssueType',
                                 'misspelledWord'
                                ]]

# rename cols
df_spell_check = df_spell_check.rename(columns={'url' : 'url_lod'})

# reset index
df_spell_check = df_spell_check.reset_index().drop(columns='index')

# Export

In [ ]:
filename_excel_export

In [ ]:
df_spell_check.to_excel('data_dumps\\' + filename_excel_export, index=False)

# Check for similar names

In [ ]:
# https://stackoverflow.com/questions/52631291/vectorizing-or-speeding-up-fuzzywuzzy-string-matching-on-pandas-column

import pandas as pd 
import numpy as np


# df = pd.DataFrame([['cliftonlarsonallen llp minneapolis MN'],
#         ['loeb and troper llp newyork NY'],
#         ["dauby o'connor and zaleski llc carmel IN"],
#         ['wegner cpas llp madison WI']],
#         columns=['org_name'])

name_vals = results_df['name.value'].to_list()

# name_vals = name_vals[0:10]

threshold = 90

def find_match(x):
    ''''''
    match = process.extract(x, name_vals, limit=2, scorer=fuzz.partial_token_sort_ratio)
#     match = match if match[1] > threshold else np.nan
    return match

# results_df['match_found'] = results_df['name.value'].progress_apply(find_match)

In [ ]:
from fuzzywuzzy import process, fuzz

In [ ]:
%%timeit

# create list
name_vals = results_df['name.value'].to_list()

name_vals = name_vals[0:5]

#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in tqdm(name_vals)
             for i in process.extract(x, name_vals, scorer=fuzz.token_sort_ratio)]

In [ ]:
#Create a dataframe from the tuples
df_similarity = pd.DataFrame(score_sort, columns=['artist','match_sort','similarity_score'])

# df_similarity = df_similarity[(df_similarity['score_sort'] > 91) &
#                               (df_similarity['score_sort'] != 100)]

# # create back up filename for a pickle
# time_stamp = time.strftime('%Y%m%d-%H%M%S')
# filename_df_similarity = f'{time_stamp}_df_similarity.xlsx'

# # export
# df_similarity.to_excel('data_dumps\\' + filename_df_similarity, index=False)

# Other options for spell-checks

In [ ]:
states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']
regex = re.compile(r'\b(' + '|'.join(states) + r')\b', re.IGNORECASE)

states2 = ['I.A.', 'K.S.', 'U.T.', 'V.A.', 'N.C.', 'N.E.', 'S.D.', 'A.L.', 'I.D.', 'F.M.', 'D.E.', 'A.K.', 'C.T.',
           'P.R.', 'N.M.', 'M.S.', 'P.W.', 'C.O.', 'N.J.', 'F.L.', 'M.N.', 'V.I.', 'N.V.', 'A.Z.', 'W.I.', 'N.D.', 
           'P.A.', 'O.K.', 'K.Y.', 'R.I.', 'N.H.', 'M.O.', 'M.E.', 'V.T.', 'G.A.', 'G.U.', 'A.S.', 'N.Y.', 'C.A.', 
           'H.I.', 'I.L.', 'T.N.', 'M.A.', 'O.H.', 'M.D.', 'M.I.', 'W.Y.', 'W.A.', 'O.R.', 'M.H.', 'S.C.', 'I.N.', 
           'L.A.', 'M.P.', 'D.C.', 'M.T.', 'A.R.', 'W.V.', 'T.X.']
regex2 = re.compile(r'\b(' + '|'.join(states) + r')\b', re.IGNORECASE)


In [ ]:
def state_finder(string:str) -> list:
    ''''''
    states2 = []
#     states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 
#               'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 
#               'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 
#               'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']

    states = ['I.A.', 'K.S.', 'U.T.', 'V.A.', 'N.C.', 'N.E.', 'S.D.', 'A.L.', 'I.D.', 'F.M.', 'D.E.', 'A.K.', 'C.T.',
           'P.R.', 'N.M.', 'M.S.', 'P.W.', 'C.O.', 'N.J.', 'F.L.', 'M.N.', 'V.I.', 'N.V.', 'A.Z.', 'W.I.', 'N.D.', 
           'P.A.', 'O.K.', 'K.Y.', 'R.I.', 'N.H.', 'M.O.', 'M.E.', 'V.T.', 'G.A.', 'G.U.', 'A.S.', 'N.Y.', 'C.A.', 
           'H.I.', 'I.L.', 'T.N.', 'M.A.', 'O.H.', 'M.D.', 'M.I.', 'W.Y.', 'W.A.', 'O.R.', 'M.H.', 'S.C.', 'I.N.', 
           'L.A.', 'M.P.', 'D.C.', 'M.T.', 'A.R.', 'W.V.', 'T.X.']                   
                  
    regex = re.compile(r'\b(' + '|'.join(states) + r')\b', re.IGNORECASE)

    try:
        string = str(string)
        states2 = re.findall(regex , string)
        return states2
    except:
        pass

In [ ]:

tqdm.pandas(desc="power DataFrame 1M to 100 random int!")
results_df['test'] = results_df['bio.value'].progress_apply(state_finder)

In [ ]:
results_df[(results_df['test'].str.len() > 2)]

In [ ]:
results_df[
#     (results_df['bio.value'].str.contains('I.A.') == True) |
    (results_df['bio.value'].str.contains('Calif\.') == True) |
    (results_df['bio.value'].str.contains('CA') == True) |
    (results_df['bio.value'].str.contains('C\.A\.') == True) |
    (results_df['bio.value'].str.contains('California') == True)]
# (results_df['bio.value'].str.contains('K.S.') == True) |
# (results_df['bio.value'].str.contains('U.T.') == True) |
# (results_df['bio.value'].str.contains('V.A.') == True) |
# (results_df['bio.value'].str.contains('N.C.') == True) |
# (results_df['bio.value'].str.contains('N.E.') == True) |
# (results_df['bio.value'].str.contains('S.D.') == True) |
# (results_df['bio.value'].str.contains('A.L.') == True) |
# (results_df['bio.value'].str.contains('I.D.') == True) |
# (results_df['bio.value'].str.contains('F.M.') == True) |
# (results_df['bio.value'].str.contains('D.E.') == True) |
# (results_df['bio.value'].str.contains('A.K.') == True) |
# (results_df['bio.value'].str.contains('C.T.') == True) |
# (results_df['bio.value'].str.contains('P.R.') == True) |
# (results_df['bio.value'].str.contains('N.M.') == True) |
# (results_df['bio.value'].str.contains('M.S.') == True) |
# (results_df['bio.value'].str.contains('P.W.') == True) |
# (results_df['bio.value'].str.contains('C.O.') == True) |
# (results_df['bio.value'].str.contains('N.J.') == True) |
# (results_df['bio.value'].str.contains('F.L.') == True) |
# (results_df['bio.value'].str.contains('M.N.') == True) |
# (results_df['bio.value'].str.contains('V.I.') == True) |
# (results_df['bio.value'].str.contains('N.V.') == True) |
# (results_df['bio.value'].str.contains('A.Z.') == True) |
# (results_df['bio.value'].str.contains('W.I.') == True) |
# (results_df['bio.value'].str.contains('N.D.') == True) |
# (results_df['bio.value'].str.contains('P.A.') == True) |
# (results_df['bio.value'].str.contains('O.K.') == True) |
# (results_df['bio.value'].str.contains('K.Y.') == True) |
# (results_df['bio.value'].str.contains('R.I.') == True) |
# (results_df['bio.value'].str.contains('N.H.') == True) |
# (results_df['bio.value'].str.contains('M.O.') == True) |
# (results_df['bio.value'].str.contains('M.E.') == True) |
# (results_df['bio.value'].str.contains('V.T.') == True) |
# (results_df['bio.value'].str.contains('G.A.') == True) |
# (results_df['bio.value'].str.contains('G.U.') == True) |
# (results_df['bio.value'].str.contains('A.S.') == True) |
# (results_df['bio.value'].str.contains('N.Y.') == True) |
# (results_df['bio.value'].str.contains('C.A.') == True) |
# (results_df['bio.value'].str.contains('H.I.') == True) |
# (results_df['bio.value'].str.contains('I.L.') == True) |
# (results_df['bio.value'].str.contains('T.N.') == True) |
# (results_df['bio.value'].str.contains('M.A.') == True) |
# (results_df['bio.value'].str.contains('O.H.') == True) |
# (results_df['bio.value'].str.contains('M.D.') == True) |
# (results_df['bio.value'].str.contains('M.I.') == True) |
# (results_df['bio.value'].str.contains('W.Y.') == True) |
# (results_df['bio.value'].str.contains('W.A.') == True) |
# (results_df['bio.value'].str.contains('O.R.') == True) |
# (results_df['bio.value'].str.contains('M.H.') == True) |
# (results_df['bio.value'].str.contains('S.C.') == True) |
# (results_df['bio.value'].str.contains('I.N.') == True) |
# (results_df['bio.value'].str.contains('L.A.') == True) |
# (results_df['bio.value'].str.contains('M.P.') == True) |
# (results_df['bio.value'].str.contains('D.C.') == True) |
# (results_df['bio.value'].str.contains('M.T.') == True) |
# (results_df['bio.value'].str.contains('A.R.') == True) |
# (results_df['bio.value'].str.contains('W.V.') == True) |
# (results_df['bio.value'].str.contains('T.X.') == True)]


In [ ]:
results_df[
#     (results_df
#             ['bio.value'].str.contains('Alabama') == True) | 
# (results_df['bio.value'].str.contains('Alaska') == True) | 
# (results_df['bio.value'].str.contains('Arizona') == True) | 
# (results_df['bio.value'].str.contains('Arkansas') == True) | 
# (results_df['bio.value'].str.contains('California') == True) | 
# (results_df['bio.value'].str.contains('Colorado') == True) | 
# (results_df['bio.value'].str.contains('Connecticut') == True) | 
# (results_df['bio.value'].str.contains('Delaware') == True) | 
# (results_df['bio.value'].str.contains('Florida') == True) | 
# (results_df['bio.value'].str.contains('Georgia') == True) | 
# (results_df['bio.value'].str.contains('Hawaii') == True) | 
# (results_df['bio.value'].str.contains('Idaho') == True) | 
# (results_df['bio.value'].str.contains('Illinois') == True) | 
# (results_df['bio.value'].str.contains('Indiana') == True) | 
# (results_df['bio.value'].str.contains('Iowa') == True) | 
# (results_df['bio.value'].str.contains('Kansas') == True) | 
# (results_df['bio.value'].str.contains('Kentucky') == True) | 
# (results_df['bio.value'].str.contains('Louisiana') == True) | 
# (results_df['bio.value'].str.contains('Maine') == True) | 
# (results_df['bio.value'].str.contains('Maryland') == True) | 
# (results_df['bio.value'].str.contains('Massachusetts') == True) | 
# (results_df['bio.value'].str.contains('Michigan') == True) | 
# (results_df['bio.value'].str.contains('Minnesota') == True) | 
# (results_df['bio.value'].str.contains('Mississippi') == True) | 
# (results_df['bio.value'].str.contains('Missouri') == True) | 
# (results_df['bio.value'].str.contains('Montana') == True) | 
# (results_df['bio.value'].str.contains('Nebraska') == True) | 
# (results_df['bio.value'].str.contains('Nevada') == True) | 
# (results_df['bio.value'].str.contains('New Hampshire') == True) | 
# (results_df['bio.value'].str.contains('New Jersey') == True) | 
# (results_df['bio.value'].str.contains('New Mexico') == True) | 
# (results_df['bio.value'].str.contains('New York') == True) | 
# (results_df['bio.value'].str.contains('North Carolina') == True) | 
# (results_df['bio.value'].str.contains('North Dakota') == True) | 
# (results_df['bio.value'].str.contains('Ohio') == True) | 
# (results_df['bio.value'].str.contains('Oklahoma') == True) | 
# (results_df['bio.value'].str.contains('Oregon') == True) | 
# (results_df['bio.value'].str.contains('Pennsylvania') == True) | 
# (results_df['bio.value'].str.contains('Rhode Island') == True) | 
# (results_df['bio.value'].str.contains('South Carolina') == True) | 
# (results_df['bio.value'].str.contains('South Dakota') == True) | 
# (results_df['bio.value'].str.contains('Tennessee') == True) | 
# (results_df['bio.value'].str.contains('Texas') == True) | 
# (results_df['bio.value'].str.contains('Utah') == True) | 
# (results_df['bio.value'].str.contains('Vermont') == True) | 
# (results_df['bio.value'].str.contains('Virginia') == True) | 
# (results_df['bio.value'].str.contains('Washington') == True) | 
# (results_df['bio.value'].str.contains('West Virginia') == True) | 
# (results_df['bio.value'].str.contains('Wisconsin') == True) | 
# (results_df['bio.value'].str.contains('Wyoming') == True) | 
# (results_df['bio.value'].str.contains('District of Columbia') == True) | 
# (results_df['bio.value'].str.contains('Guam') == True) | 
# (results_df['bio.value'].str.contains('Marshall Islands') == True) | 
# (results_df['bio.value'].str.contains('Northern Mariana Island') == True) | 
# (results_df['bio.value'].str.contains('Puerto Rico') == True) | 
# (results_df['bio.value'].str.contains('Virgin Islands') == True) | 
# (results_df['bio.value'].str.contains('AL') == True) | 
# (results_df['bio.value'].str.contains('AK') == True) | 
# (results_df['bio.value'].str.contains('AZ') == True) | 
# (results_df['bio.value'].str.contains('AR') == True) | 
# (results_df['bio.value'].str.contains('CA') == True) | 
# (results_df['bio.value'].str.contains('CO') == True) | 
# (results_df['bio.value'].str.contains('CT') == True) | 
# (results_df['bio.value'].str.contains('DE') == True) | 
# (results_df['bio.value'].str.contains('FL') == True) | 
# (results_df['bio.value'].str.contains('GA') == True) | 
# (results_df['bio.value'].str.contains('HI') == True) | 
# (results_df['bio.value'].str.contains('ID') == True) | 
# (results_df['bio.value'].str.contains('IL') == True) | 
# (results_df['bio.value'].str.contains('IN') == True) | 
# (results_df['bio.value'].str.contains('IA') == True) | 
# (results_df['bio.value'].str.contains('KS') == True) | 
# (results_df['bio.value'].str.contains('KY') == True) | 
# (results_df['bio.value'].str.contains('LA') == True) | 
# (results_df['bio.value'].str.contains('ME') == True) | 
# (results_df['bio.value'].str.contains('MD') == True) | 
# (results_df['bio.value'].str.contains('MA') == True) | 
# (results_df['bio.value'].str.contains('MI') == True) | 
# (results_df['bio.value'].str.contains('MN') == True) | 
# (results_df['bio.value'].str.contains('MS') == True) | 
# (results_df['bio.value'].str.contains('MO') == True) | 
# (results_df['bio.value'].str.contains('MT') == True) | 
# (results_df['bio.value'].str.contains('NE') == True) | 
# (results_df['bio.value'].str.contains('NV') == True) | 
# (results_df['bio.value'].str.contains('NH') == True) | 
# (results_df['bio.value'].str.contains('NJ') == True) | 
# (results_df['bio.value'].str.contains('NM') == True) | 
# (results_df['bio.value'].str.contains('NY') == True) | 
# (results_df['bio.value'].str.contains('NC') == True) | 
# (results_df['bio.value'].str.contains('ND') == True) | 
# (results_df['bio.value'].str.contains('OH') == True) | 
# (results_df['bio.value'].str.contains('OK') == True) | 
# (results_df['bio.value'].str.contains('OR') == True) | 
# (results_df['bio.value'].str.contains('PA') == True) | 
# (results_df['bio.value'].str.contains('RI') == True) | 
# (results_df['bio.value'].str.contains('SC') == True) | 
# (results_df['bio.value'].str.contains('SD') == True) | 
# (results_df['bio.value'].str.contains('TN') == True) | 
(results_df['bio.value'].str.contains('TX') == True) | 
(results_df['bio.value'].str.contains('T.X.') == True) | 
# (results_df['bio.value'].str.contains('UT') == True) | 
# (results_df['bio.value'].str.contains('VT') == True) | 
# (results_df['bio.value'].str.contains('VA') == True) | 
# (results_df['bio.value'].str.contains('WA') == True) | 
# (results_df['bio.value'].str.contains('WV') == True) | 
# (results_df['bio.value'].str.contains('WI') == True) | 
# (results_df['bio.value'].str.contains('WY') == True) | 
# (results_df['bio.value'].str.contains('DC') == True) | 
# (results_df['bio.value'].str.contains('GU') == True) | 
# (results_df['bio.value'].str.contains('MH') == True) | 
# (results_df['bio.value'].str.contains('MP') == True) | 
# (results_df['bio.value'].str.contains('PR') == True) | 
# (results_df['bio.value'].str.contains('VI') == True) | 
# (results_df['bio.value'].str.contains(' Ala.') == True) | 
# (results_df['bio.value'].str.contains(' Alaska') == True) | 
# (results_df['bio.value'].str.contains(' Ariz.') == True) | 
# (results_df['bio.value'].str.contains(' Ark.') == True) | 
# (results_df['bio.value'].str.contains(' Calif.') == True) | 
# (results_df['bio.value'].str.contains(' Color.') == True) | 
# (results_df['bio.value'].str.contains(' Conn.') == True) | 
# (results_df['bio.value'].str.contains(' Del.') == True) | 
# (results_df['bio.value'].str.contains(' Fla.') == True) | 
# (results_df['bio.value'].str.contains(' Ga.') == True) | 
# (results_df['bio.value'].str.contains(' Hawaii') == True) | 
# (results_df['bio.value'].str.contains(' Idaho') == True) | 
# (results_df['bio.value'].str.contains(' Ill.') == True) | 
# (results_df['bio.value'].str.contains(' Ind.') == True) | 
# (results_df['bio.value'].str.contains(' Iowa') == True) | 
# (results_df['bio.value'].str.contains(' Kan.') == True) | 
# (results_df['bio.value'].str.contains(' Ky.') == True) | 
# (results_df['bio.value'].str.contains(' La.') == True) | 
# (results_df['bio.value'].str.contains(' Maine') == True) | 
# (results_df['bio.value'].str.contains(' Md.') == True) | 
# (results_df['bio.value'].str.contains(' Mass.') == True) | 
# (results_df['bio.value'].str.contains(' Mich.') == True) | 
# (results_df['bio.value'].str.contains(' Minn.') == True) | 
# (results_df['bio.value'].str.contains(' Miss.') == True) | 
# (results_df['bio.value'].str.contains(' Mo.') == True) | 
# (results_df['bio.value'].str.contains(' Mont.') == True) | 
# (results_df['bio.value'].str.contains(' Neb.') == True) | 
# (results_df['bio.value'].str.contains(' Nev.') == True) | 
# (results_df['bio.value'].str.contains(' N.H.') == True) | 
# (results_df['bio.value'].str.contains(' N.J.') == True) | 
# (results_df['bio.value'].str.contains(' N.M.') == True) | 
# (results_df['bio.value'].str.contains(' N.Y.') == True) | 
# (results_df['bio.value'].str.contains(' N.C.') == True) | 
# (results_df['bio.value'].str.contains(' N.D.') == True) | 
# (results_df['bio.value'].str.contains(' Ohio') == True) | 
# (results_df['bio.value'].str.contains(' Okla.') == True) | 
# (results_df['bio.value'].str.contains(' Ore.') == True) | 
# (results_df['bio.value'].str.contains(' Pa.') == True) | 
# (results_df['bio.value'].str.contains(' R.I.') == True) | 
# (results_df['bio.value'].str.contains(' S.C.') == True) | 
# (results_df['bio.value'].str.contains(' S.Dak.') == True) | 
# (results_df['bio.value'].str.contains(' Tenn.') == True) | 
(results_df['bio.value'].str.contains(' Tex.') == True)  ]
# (results_df['bio.value'].str.contains(' Utah') == True) | 
# (results_df['bio.value'].str.contains(' V.T.') == True) | 
# (results_df['bio.value'].str.contains(' Va.') == True) | 
# (results_df['bio.value'].str.contains(' Wash.') == True) | 
# (results_df['bio.value'].str.contains(' W.Va.') == True) | 
# (results_df['bio.value'].str.contains(' Wis.') == True) | 
# (results_df['bio.value'].str.contains(' Wyo.') == True) | 
# (results_df['bio.value'].str.contains(' D.C.') == True) | 
# (results_df['bio.value'].str.contains(' Guam') == True) | 
# (results_df['bio.value'].str.contains(' M.I.') == True) | 
# (results_df['bio.value'].str.contains(' CNMI') == True) | 
# (results_df['bio.value'].str.contains(' P.R. or PUR') == True) | 
# (results_df['bio.value'].str.contains(' V.I.') == True) ]

In [ ]:
results_df['test'] = results_df['bio.value'].str.extract(r'(?!BC|CE)(A-Z)')

In [ ]:
results_df[(results_df['test'].notnull())]

In [ ]:
results_df[(results_df['bio.value'].str.contains('Tex') == True)].head(2)

In [ ]:
results_df.head()

In [ ]:
no_upper['misspelledWord'].value_counts()

In [ ]:
no_upper['misspelledWord'].iloc[26:27]

In [ ]:
df_errors['test'] = df_errors['context'].astype(str).str.extract('([\s]+-[\s]+)')

In [ ]:
df_errors[(df_errors['test'].notnull())].head(10)